In [1]:
import qrcode
import os
from PIL import Image, ImageDraw, ImageFont

def generate_qr_codes_2(text_list, output_dir="qr_codes_4"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_paths = []
    for idx, text in enumerate(text_list):
        # --- Generate QR code and ensure a proper PIL Image mode ---
        qr = qrcode.make(text, border=2, box_size=15)
        qr = qr.convert("RGBA")  # ensure consistent mode and alpha channel if present
        qr_width, qr_height = qr.size

        # --- Choose font (default system font if custom unavailable) ---
        try:
            font = ImageFont.truetype("montserrat.ttf", 40)
        except Exception:
            font = ImageFont.load_default(20)

        # --- Measure text size (compat for older Pillow) ---
        dummy_img = Image.new("RGB", (1, 1))
        draw = ImageDraw.Draw(dummy_img)
        if hasattr(draw, "textbbox"):
            bbox = draw.textbbox((0, 0), text, font=font)
            text_width = bbox[2] - bbox[0]
            text_height = bbox[3] - bbox[1]
        else:
            text_width, text_height = draw.textsize(text, font=font)

        # --- Add padding and new image height for text ---
        padding = 20
        total_height = qr_height + text_height + padding

        # --- Create final image with white background ---
        final_img = Image.new("RGB", (qr_width, total_height), "white")
        draw = ImageDraw.Draw(final_img)

        # --- Draw text centered at top ---
        text_x = max((qr_width - text_width) // 2, 0)
        text_y = 10  # margin from top
        draw.text((text_x, text_y), text, fill="black", font=font)

        # --- Paste QR below text using a 4-item box and alpha mask if present ---
        qr_y = text_y + text_height + 5
        box = (0, qr_y, qr_width, qr_y + qr_height)
        alpha = qr.split()[3] if qr.mode == "RGBA" else None
        final_img.paste(qr.convert("RGB"), box, mask=alpha)

        # --- Save final image safely ---
        safe_text = "".join(c if c.isalnum() else "_" for c in text).strip("_")
        if not safe_text:
            safe_text = f"qr_{idx}"
        safe_text = safe_text[:200]
        file_path = os.path.join(output_dir, f"{safe_text}.png")
        final_img.save(file_path)
        image_paths.append(file_path)

    return image_paths
# ...existing code...

In [4]:

start_num = 1301
count = 699  # or any number of matric numbers you want to generate

matric_numbers = [f"DU{num:04d}" for num in range(start_num, start_num + count + 1)]
print(matric_numbers)

['DU1301', 'DU1302', 'DU1303', 'DU1304', 'DU1305', 'DU1306', 'DU1307', 'DU1308', 'DU1309', 'DU1310', 'DU1311', 'DU1312', 'DU1313', 'DU1314', 'DU1315', 'DU1316', 'DU1317', 'DU1318', 'DU1319', 'DU1320', 'DU1321', 'DU1322', 'DU1323', 'DU1324', 'DU1325', 'DU1326', 'DU1327', 'DU1328', 'DU1329', 'DU1330', 'DU1331', 'DU1332', 'DU1333', 'DU1334', 'DU1335', 'DU1336', 'DU1337', 'DU1338', 'DU1339', 'DU1340', 'DU1341', 'DU1342', 'DU1343', 'DU1344', 'DU1345', 'DU1346', 'DU1347', 'DU1348', 'DU1349', 'DU1350', 'DU1351', 'DU1352', 'DU1353', 'DU1354', 'DU1355', 'DU1356', 'DU1357', 'DU1358', 'DU1359', 'DU1360', 'DU1361', 'DU1362', 'DU1363', 'DU1364', 'DU1365', 'DU1366', 'DU1367', 'DU1368', 'DU1369', 'DU1370', 'DU1371', 'DU1372', 'DU1373', 'DU1374', 'DU1375', 'DU1376', 'DU1377', 'DU1378', 'DU1379', 'DU1380', 'DU1381', 'DU1382', 'DU1383', 'DU1384', 'DU1385', 'DU1386', 'DU1387', 'DU1388', 'DU1389', 'DU1390', 'DU1391', 'DU1392', 'DU1393', 'DU1394', 'DU1395', 'DU1396', 'DU1397', 'DU1398', 'DU1399', 'DU1400',

In [2]:
image_files = generate_qr_codes_2(["DU1249","DU1269"], "qr_codes_5")
print("Generated QR code images:", image_files)

Generated QR code images: ['qr_codes_5\\DU1249.png', 'qr_codes_5\\DU1269.png']


In [3]:
import math

def arrange_qr_codes_on_a4(input_dir, output_dir="qr_sheets3", margin=10, padding=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # A4 size in pixels for 300 DPI
    A4_WIDTH, A4_HEIGHT = 2480, 3508  

    # --- Load all image paths ---
    image_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir)
                   if f.lower().endswith((".png", ".jpg", ".jpeg"))]
    if not image_files:
        print("No QR code images found in the folder.")
        return []

    # --- Load first image to determine size ---
    sample_img = Image.open(image_files[0])
    img_w, img_h = sample_img.size
    sample_img.close()

    # --- Compute how many fit per row/column ---
    cols = max(1, (A4_WIDTH - 2 * margin + padding) // (img_w + padding))
    rows = max(1, (A4_HEIGHT - 2 * margin + padding) // (img_h + padding))
    per_page = cols * rows

    print(f"Fitting {cols} columns × {rows} rows = {per_page} per sheet")

    pages = []
    total_pages = math.ceil(len(image_files) / per_page)

    for page_num in range(total_pages):
        page = Image.new("RGB", (A4_WIDTH, A4_HEIGHT), "white")
        draw = ImageDraw.Draw(page)

        start_idx = page_num * per_page
        end_idx = start_idx + per_page
        subset = image_files[start_idx:end_idx]

        for i, path in enumerate(subset):
            img = Image.open(path)
            col = i % cols
            row = i // cols

            x = margin + col * (img_w + padding)
            y = margin + row * (img_h + padding)
            page.paste(img, (x, y))
            img.close()

        # --- Save sheet ---
        output_path = os.path.join(output_dir, f"qr_sheet_{page_num + 1}.png")
        page.save(output_path, "PNG")
        pages.append(output_path)
        print(f"Saved: {output_path}")

    print(f"✅ Done! Created {len(pages)} A4 sheet(s).")
    return pages


# Example usage:
# arrange_qr_codes_on_a4("qr_codes_2")


In [4]:
arrange_qr_codes_on_a4("qr_codes_5")

Fitting 6 columns × 8 rows = 48 per sheet
Saved: qr_sheets3\qr_sheet_1.png
✅ Done! Created 1 A4 sheet(s).


['qr_sheets3\\qr_sheet_1.png']

In [8]:
import re
import os

def _numeric_sort_key(path):
    name = os.path.basename(path)
    m = re.search(r'(\d+)', name)
    return int(m.group(1)) if m else name.lower()

def images_to_single_pdf(input_dir=None, image_list=None, output_pdf="qr_codes.pdf"):
    """
    Create a single PDF from images. Provide either image_list (full paths) or input_dir.
    """
    if image_list is None:
        if input_dir is None:
            raise ValueError("Provide either input_dir or image_list")
        image_list = [os.path.join(input_dir, f) for f in os.listdir(input_dir)
                      if f.lower().endswith((".png", ".jpg", ".jpeg"))]

    if not image_list:
        print("No images found to combine.")
        return None

    # sort naturally by numeric part if present
    image_list = sorted(image_list, key=_numeric_sort_key)

    # open and convert images to RGB
    pil_images = []
    for p in image_list:
        try:
            img = Image.open(p).convert("RGB")
            pil_images.append(img)
        except Exception as e:
            print(f"Skipping {p}: {e}")

    if not pil_images:
        print("No valid images to save.")
        return None

    first, rest = pil_images[0], pil_images[1:]
    first.save(output_pdf, save_all=True, append_images=rest)
    # close all images
    for im in pil_images:
        im.close()

    print(f"Saved PDF: {output_pdf}")
    return output_pdf


In [9]:
images_to_single_pdf(input_dir="qr_sheets2", output_pdf="qr_codes_4.pdf")

Saved PDF: qr_codes_4.pdf


'qr_codes_4.pdf'